In [ ]:
import numpy as np
import pandas as pd
import quantecon as qe
from ast import literal_eval
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def creating_samples(samples_number, element):
    gen=0
    uniform_number = len(element)
    uniform_samples = (-qe.simplex_grid(len(element), 1)+1)/(len(element)-1)
    if samples_number > uniform_number:
        random_samples = np.random.rand((samples_number-uniform_number), len(element))
        for i in range(samples_number-uniform_number):
            random_samples[i] = np.around(random_samples[i]/sum(random_samples[i]), decimals = 3)
        sample = np.vstack((uniform_samples, random_samples))
    else:
        sample = uniform_samples[:samples_number]
    samples = []
    for i in range(samples_number):
        samples.append(list(sample[i]))
    samples = np.array(samples)
    generation = []
    for i in range (samples_number):
            generation.append(gen)

    ID = np.arange(1, (samples_number+1))

    data = {'ID' : ID, 'Elements': [element], 'Generation': generation}
    df = pd.DataFrame(data=data, index = np.arange(samples_number))
    df = pd.concat([df, pd.DataFrame(([[i] for i in samples]), columns = ['Position'])], axis = 1)
    df.to_csv("Result/Initial Population.txt", sep='\t', index=False, mode='w')
    return df

In [ ]:
#Loading data and fitting
df_old = pd.read_csv('Data/AuIrOsPdPtReRhRu_0.60_compositions_and_targets.csv')
X_columns_old = ['Pt','Pd','Au','Ru','Rh','Ir','Re','Os']
x_old = df_old[X_columns_old].to_numpy()
y_old = df_old['current_over_capacitance'].to_numpy()

#Rndom Forest Regression
reg = RandomForestRegressor(n_estimators = 1024,
                           bootstrap = True,
                           max_depth = None,
                           max_features = 'auto',
                           min_samples_leaf = 1,
                           min_samples_split = 2,
                           oob_score = True)
reg = reg.fit(x_old, y_old)

In [ ]:
#Making Class
class pso:
    def __init__ (self, data, step, target = np.array([0.333, 0.333, 0.333])):
        self.datalog = data
        #Reading "Elements" columns from string to list
        self.datalog['Elements'] = self.datalog["Elements"].apply(lambda x: literal_eval(x))
        
        #Select latest generation
        self.generation = self.datalog['Generation'].max()
        
        #Creating np.array of "Position" column and dropping the string type "Position" column
        self.position = []
        for i in range(self.datalog['ID'].max()):
            self.position.append(list(np.fromstring(self.datalog['Position'][i][1:-1], dtype=float, sep=' ')))
        self.position = np.array(self.position)
        self.datalog = self.datalog.drop(columns=['Position'])
        self.datalog = pd.concat([self.datalog, pd.DataFrame(([[i] for i in self.position]), columns = ['Position'])], axis = 1)
        
        #Creating "Velocity" column for the "0" generation
        if self.generation == 0:
            self.velocity = np.around((-(self.position - target)/step), decimals = 3)
            self.datalog = pd.concat([self.datalog, pd.DataFrame(([[i] for i in self.velocity]), columns = ['Velocity'])], axis = 1)
        
        #Creating blank "Activity" column
        self.datalog = pd.concat([self.datalog, pd.DataFrame(columns = ['Activity'], index = np.arange(self.datalog['ID'].max()))], axis = 1)      

        #Filling the "Activity" column with RFR
        self.f_activity(self.position, self.datalog)
            
        #Creating dataframe of the latest generation
        self.working_generation = self.datalog.loc[self.datalog['Generation']==self.generation]
        

    def move(self, size=1):
        #Performing move function for certain number of times    
        self.generation += 1
        self.working_generation['Generation'] += 1 
        
        #Creating array of "Position" column
        self.position_now = []
        for i in range(self.working_generation['ID'].max()):
            self.position_now.append(list(self.working_generation['Position'][i]))
        self.position_now = np.array(self.position_now)
        
        #Moving process
        self.position_now = np.around((self.position_now + self.velocity*size), decimals = 3)
        for i in range(self.working_generation['ID'].max()):
            """if self.position_now[i].max() > 1 or self.position_now[i].min() < 0:
                self.position_now[i] -= 2 * self.velocity[i]"""          
            while self.position_now[i].max() > 1 or self.position_now[i].min() < 0:
                self.position_now[i] = np.around((self.position_now[i]-self.velocity[i]), decimals = 3)
            
        #Creating dataframe
        self.working_generation = self.working_generation.drop(columns=['Position'])
        self.working_generation = pd.concat([self.working_generation, pd.DataFrame(([[i] for i in self.position_now]), columns = ['Position'])], axis = 1)
        

        """
        self.working_generation[]
        for i in range(len(self.working_generation)):
            if self.working_generation["Position"][i].max() > 1 or self.working_generation["Position"][i].min() < 0:
                self.working_generation["Position"][i] -= 2*self.working_generation['Velocity'][i]*size"""
                
        
        #Filling the "Activity" column with RFR
        self.f_activity(self.position_now, self.working_generation)
        
        #Concating the tables
        self.datalog = pd.concat([self.datalog, self.working_generation])
        self.store_datalog()
        return
    
    def f_activity(self, x, dataframe):
        global reg
        a = [0, 0, 0, 0, 0]
        self.dummy = []
        for i in range(dataframe['ID'].max()):
            self.dummy.append(np.hstack((x[i], a)))
        self.activity = []
        for i in range (dataframe['ID'].max()):
            self.activity.append(float(reg.predict(np.reshape(self.dummy[i], (1, -1)))))
        for i in range(dataframe['ID'].max()):
            dataframe['Activity'][i] = self.activity[i]    
        return

    def store_datalog(self):
        self.datalog.to_csv("Result/Initial Population_gen "+str(self.generation)+".txt", sep='\t', index=False, mode='w')
        return
  

In [ ]:
samples_number = 6
element = ['Pt', 'Pd', 'Au']
creating_samples(samples_number, element)

In [ ]:
step = 5
target = np.array([0.9, 0.1, 0])
population = pso(pd.read_csv('Result/Initial Population.txt', sep='\t'), step, target)

In [ ]:
population.datalog

In [ ]:
population.move(8)